In [92]:
import pandas as pd

In [123]:
#distance between two points in the interval [0,1] the "long way" (ie, distance moving from b
#to a passing through 1.0 (assuming b > a). 

def wrapDist(a, b):
    return (1.0-b)+a

#decide whether points in an orbit are contained ina  semi-circle (which makes it Sturmian)
#here this is decided by conjugate orbit as rationals in the unit interval [0,1]
#and seeing if they are contained within a closed interval (mod 1) of length 
#less than .5

def isSturmian(orbit, denom):
    ascOrbit = sorted(orbit)
    #print(ascOrbit)
    spread = 1.0
    period = len(ascOrbit)
    
    spread = (ascOrbit[period-1] - ascOrbit[0])/denom 
    
    if spread <= .5:
        return True
    else:
        for i in range(len(ascOrbit)):
            spread = wrapDist(((ascOrbit[i])/denom), (ascOrbit[(i+1)%(period)])/denom)
            if spread <=.5:
                return True 
        return False

In [143]:
#function caculates the periodic points of the doubling map
#that have period n

def calc_pOrbits(n): 
    orbit = [] #single list for a single orbit
    orbitList = [] #this will be a list of orbits (list of lists)
    pp_denom = 2**n - 1
    
    num_points = pp_denom
    num_points_so_far = 0
    
    num_list = set([]) #store elements in a set to guarentee uniqueness
    
    for x in range(0, num_points-1, n):
        pp_num = 1
        while pp_num in num_list: #only try numerators that have not been used yet
            pp_num += 1
            
        num_list.add(pp_num)
        
        orbit_start = pp_num
        orbit = []
        orbit.append(orbit_start)
        num_points_so_far +=1
        
        for y in range(1, n): 
            orbit.append((pp_num*2)%pp_denom)
            pp_num = (pp_num*2)%pp_denom
            num_list.add(pp_num)
            num_points_so_far +=1
      
        orbitSet = set(orbit)
        
        #identify duplicate orbits... ie, we'll find orbits of lower period 
        #in higher periods. ie, we might have period 2 orbits appearing as period 4 orbits
        #here we filter out if there are non unique points in the "orbit"
        
        if (len(orbitSet) != len(orbit)):
            #orbit = ["DUPE"]
            orbit.insert(0, "DUPE") #add "DUPE" in the dataframe to indicate duplicate orbit
        
        else: 
            orbit.insert(0, isSturmian(orbit, pp_denom))
        orbit.insert(0, pp_denom)
        orbit.insert(0, n)
        
        orbitList.append(orbit)   
        
    return orbitList

In [149]:
calc_pOrbits(3) #testing function. first three items in list are "period," "denom," and "sturmian?"

[[3, 7, True, 1, 2, 4], [3, 7, True, 3, 6, 5]]

In [139]:
#push a list of orbits of a given period into a pandas dataframe
def orbitChart(n):
    orbitN = calc_pOrbits(n)
  
    orbitTable = pd.DataFrame(calc_pOrbits(n))
    #orbitTable = orbitTable.astype(str) + '/{}'.format(2**n-1) 
    return orbitTable
    

In [150]:
test = orbitChart(5) ##testing this method
test

,0,1,2,3,4,5,6,7
0,5,31,True,1,2,4,8,16
1,5,31,False,3,6,12,24,17
2,5,31,True,5,10,20,9,18
3,5,31,False,7,14,28,25,19
4,5,31,True,11,22,13,26,21
5,5,31,True,15,30,29,27,23


In [147]:
# make a table of all orbits up between period 2 and 5
orbitList = []
for x in range(2, 6):
    orbitList.append(orbitChart(x))
orbitTab = pd.concat(orbitList, ignore_index=True)
orbitTab.rename(columns={0:'period', 1: 'denom', 2: 'Sturmian?'}, inplace=True)

#renumber/name columns to be 
for x in range(3, 12):
    orbitTab.rename(columns={x: 'f{}'.format(x-3)}, inplace=True)
orbitTab

,period,denom,Sturmian?,f0,f1,f2,f3,f4
0,2,3,True,1,2,NaN,NaN,NaN
1,3,7,True,1,2,4.0,NaN,NaN
2,3,7,True,3,6,5.0,NaN,NaN
3,4,15,True,1,2,4.0,8.0,NaN
4,4,15,False,3,6,12.0,9.0,NaN
5,4,15,DUPE,5,10,5.0,10.0,NaN
6,4,15,True,7,14,13.0,11.0,NaN
7,5,31,True,1,2,4.0,8.0,16.0
8,5,31,False,3,6,12.0,24.0,17.0
9,5,31,True,5,10,20.0,9.0,18.0


In [152]:
#function to generate a table of periodic orbits, ranging from period min to max
def OrbitTable(min, max):
    orbitList = []
    for x in range(min, max+1):
        orbitList.append(orbitChart(x))
    orbitTab = pd.concat(orbitList, ignore_index=True)
    orbitTab.rename(columns={0:'period', 1: 'denom', 2: 'sturmian'}, inplace=True)

    #renumber/name columns to be 
    for x in range(3, 12):
        orbitTab.rename(columns={x: 'f{}'.format(x-3)}, inplace=True)
    return orbitTab

In [157]:
OT = OrbitTable(1, 12) #generate table of Orbits of desired range of periods
OT

,period,denom,sturmian,f0,f1,f2,f3,f4,f5,f6,f7,f8,12,13,14
0,2,3,True,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,7,True,1,2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,7,True,3,6,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,15,True,1,2,4.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,15,False,3,6,12.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4,15,DUPE,5,10,5.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,4,15,True,7,14,13.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,5,31,True,1,2,4.0,8.0,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,5,31,False,3,6,12.0,24.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,5,31,True,5,10,20.0,9.0,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [158]:
#filter out duplicate orbits:
OT.sample(10)

,period,denom,sturmian,f0,f1,f2,f3,f4,f5,f6,f7,f8,12,13,14
190,10,1023,False,147,294,588.0,153.0,306.0,612.0,201.0,402.0,804.0,585.0,NaN,NaN
584,12,4095,False,367,734,1468.0,2936.0,1777.0,3554.0,3013.0,1931.0,3862.0,3629.0,3163.0,2231.0
69,8,255,DUPE,85,170,85.0,170.0,85.0,170.0,85.0,170.0,NaN,NaN,NaN,NaN
686,12,4095,False,749,1498,2996.0,1897.0,3794.0,3493.0,2891.0,1687.0,3374.0,2653.0,1211.0,2422.0
386,11,2047,False,445,890,1780.0,1513.0,979.0,1958.0,1869.0,1691.0,1335.0,623.0,1246.0,NaN
345,11,2047,False,255,510,1020.0,2040.0,2033.0,2019.0,1991.0,1935.0,1823.0,1599.0,1151.0,NaN
39,7,127,False,47,94,61.0,122.0,117.0,107.0,87.0,NaN,NaN,NaN,NaN,NaN
310,11,2047,False,169,338,676.0,1352.0,657.0,1314.0,581.0,1162.0,277.0,554.0,1108.0,NaN
536,12,4095,False,243,486,972.0,1944.0,3888.0,3681.0,3267.0,2439.0,783.0,1566.0,3132.0,2169.0
137,10,1023,False,13,26,52.0,104.0,208.0,416.0,832.0,641.0,259.0,518.0,NaN,NaN


In [166]:
#filter out duplicate orbits
noDupes = OT['sturmian']!='DUPE'

filteredOT = OT[noDupes]

#filter out non-sturmian orbits
filteredOT = filteredOT[filteredOT['sturmian']==True]

filteredOT


,period,denom,sturmian,f0,f1,f2,f3,f4,f5,f6,f7,f8,12,13,14
0,2,3,True,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,7,True,1,2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,7,True,3,6,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,15,True,1,2,4.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,4,15,True,7,14,13.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,5,31,True,1,2,4.0,8.0,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,5,31,True,5,10,20.0,9.0,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,5,31,True,11,22,13.0,26.0,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,5,31,True,15,30,29.0,27.0,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,6,63,True,1,2,4.0,8.0,16.0,32.0,NaN,NaN,NaN,NaN,NaN,NaN
